## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 31/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Trip service](#toc1_)    
  - [Loading the data](#toc1_1_)    
  - [Plotting the data](#toc1_2_)    
  - [Using the service](#toc1_3_)    
    - [General options](#toc1_3_1_)    
    - [Additional options](#toc1_3_2_)    
    - [Making the request](#toc1_3_3_)    
      - [No source & No destination](#toc1_3_3_1_)    
      - [Fixed source & Fixed destination](#toc1_3_3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

---

# <a id='toc1_'></a>[Trip service](#toc0_)

Use this service if you want to solve the TSP problem. Read more about it [here](https://project-osrm.org/docs/v5.24.0/api/?language=cURL#trip-service).

## <a id='toc1_1_'></a>[Loading the data](#toc0_)

Since, I am working with the `Morocco` map, I randomly selected some cities from that country to demonstrate how the `Trip` service works. If you are using another country, make sure to use your own data.

In [2]:
import pandas as pd
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


df = pd.read_csv("D:/A-HKU/OSRM_Learning/notebook/data/trip_service_data.csv")
points = []
for i, row in df.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]
    points.append(Point(latitude=latitude, longitude=longitude))

points[:5]

[Point(latitude=22.14839107, longitude=113.4477153),
 Point(latitude=22.1092719, longitude=113.4312568),
 Point(latitude=22.20557892, longitude=113.4609378),
 Point(latitude=22.23426286, longitude=113.5016648),
 Point(latitude=22.22157594, longitude=113.4440119)]

## <a id='toc1_2_'></a>[Plotting the data](#toc0_)

In [4]:
import folium

folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=12)

for point in points:
    folium.Marker(
        location=[point.latitude, point.longitude],
        icon=folium.Icon(color='blue')
    ).add_to(folium_map)

folium_map

## <a id='toc1_3_'></a>[Using the service](#toc0_)

### <a id='toc1_3_1_'></a>[General options](#toc0_)

The general options are used in every OSRM service, read more about them [here](https://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options). Here is a table that summarizes the options with their description.


| Parameter | Description |
|-----------|-------------|
| service | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| version | Version of the protocol implemented by the service. `v1` for all OSRM 5.x installations |
| profile | Mode of transportation, is determined statically by the Lua profile that is used to prepare the data using `osrm-extract`. Typically `car`, `bike` or `foot` if using one of the supplied profiles. |
| coordinates | String of format `{longitude},{latitude};{longitude},{latitude};{longitude},{latitude} ...]` or `polyline({polyline})`. |
| format | Only `json` is supported at the moment. This parameter is optional and defaults to `json`. |

### <a id='toc1_3_2_'></a>[Additional options](#toc0_)

The additional parameters available for requests to the trip service are:

- `roundtrip`: A boolean indicating whether the returned route is a roundtrip (i.e., the route returns to the starting location).
- `source`: A string specifying the starting point of the returned route, which can be either `any` or `first` coordinate.
- `destination`: A string specifying the endpoint of the returned route, which can be either `any` or `last` coordinate.

### <a id='toc1_3_3_'></a>[Making the request](#toc0_)

The trip endpoint follows this format: `/trip/v1/{profile}/{coordinates}?roundtrip={true|false}&source{any|first}&destination{any|last}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={simplified|full|false}&annotations={true|false}`.

#### <a id='toc1_3_3_1_'></a>[No source & No destination](#toc0_)

This is the default case. OSRM will choose the start and end points randonly since they are not specified in the request. Also, by default, OSRM will start and finish at the same point.

In [5]:
service = 'trip'
version = 'v1'
profile = 'cycling'
host = 'http://localhost:5000'

In [6]:
import requests

coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])
url = f'{host}/{service}/{version}/{profile}/{coordinates}'
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
}

response = requests.get(url, params=params)
response.status_code

200

The trip can be extracted from the `trip` field. OSRM will return the fastest trip.

In [9]:
data = response.json()
data.keys()

dict_keys(['code', 'trips', 'waypoints'])

This looks familiar, we have seen such an output in the first notebook where we talked about the `route` service. We will use the legs to draw the snapped route.

In [10]:
trip = data['trips'][0]
trip.keys()

dict_keys(['legs', 'weight_name', 'geometry', 'weight', 'duration', 'distance'])

Let's create some helper function to add markers to the map, draw polylines and create the HTML for the popup.

In [12]:
from folium import Map


def create_marker(
    map_obj: Map,
    point: Point,
    label: str,
    background_color: str,
    y_offset: int = 0
) -> None:
    folium.Marker(
        location=[point.latitude, point.longitude],
        icon=folium.DivIcon(html=f'''
            <div style="
                background-color: {background_color};
                border-radius: 5px;
                padding: 2px 5px;
                font-size: 8pt;
                text-align: center;
                width: 20px;
                height: 20px;
                line-height: 20px;
                transform: translate(0px, {y_offset}px);
                ">
                {label}
            </div>
        ''')
    ).add_to(map_obj)


def add_polyline(
    map_obj: Map,
    coordinates: list[Point],
    color: str,
    popup_html: str
) -> None:
    folium.PolyLine(
        locations=[[point.latitude, point.longitude] for point in coordinates],
        color=color,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(map_obj)


def get_popup_html(leg: dict, i: int) -> str:
    distance_km = leg['distance'] / 1000
    duration_sec = leg['duration']
    hours, remainder = divmod(duration_sec, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"""Leg number: {i + 1}<br>Distance: {distance_km:.2f} km<br>Duration: {int(hours):02}:{int(minutes):02}:{int(seconds):02}"""

The start point is colored in `green` while the end point is colored in `red`. As you can see, both points are the same, that is because by default OSRM will make a roundtrip, meaning it will start at a point and go back to it at the end of the trip.

In [13]:
folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=6)

colors = ["red", "black"]
legs = trip['legs']

for i, leg in enumerate(legs):
    start_coords = leg["steps"][0]["geometry"]["coordinates"][0]
    starting_point = Point(latitude=start_coords[1], longitude=start_coords[0])
    create_marker(
        map_obj=folium_map,
        point=starting_point,
        label=i+1,
        background_color="#93ff6a" if i == 0 else "#6aaeff",
        y_offset=10 if i == 0 else 0
    )

    for step in leg["steps"]:
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in step["geometry"]["coordinates"]]

        add_polyline(
            map_obj=folium_map,
            coordinates=coordinates,
            color=colors[i % len(colors)],
            popup_html=get_popup_html(leg=leg, i=i)
        )

end_coords = legs[-1]["steps"][-1]["geometry"]["coordinates"][-1]
end_point = Point(latitude=end_coords[1], longitude=end_coords[0])
create_marker(
    map_obj=folium_map,
    point=end_point,
    label=len(legs) + 1,
    background_color="#ff6a6a",
    y_offset=-10
)

folium_map

#### <a id='toc1_3_3_2_'></a>[Fixed source & Fixed destination](#toc0_)

I only added three parameters to the request `roundtrip`, `source`, and `destination`.

In [13]:
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])
url = f'{host}/{service}/{version}/{profile}/{coordinates}'
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
    "source": "first",
    "destination": "last",
    "roundtrip": "false",
}

response = requests.get(url, params=params)
data = response.json()
trip = data['trips'][0]

As you can see, the start and end points are no longer the same.

In [14]:
folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=6)

colors = ["red", "black"]
legs = trip['legs']

for i, leg in enumerate(legs):
    start_coords = leg["steps"][0]["geometry"]["coordinates"][0]
    starting_point = Point(latitude=start_coords[1], longitude=start_coords[0])
    create_marker(
        map_obj=folium_map,
        point=starting_point,
        label=i+1,
        background_color="#93ff6a" if i == 0 else "#6aaeff",
        y_offset=0
    )

    for step in leg["steps"]:
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in step["geometry"]["coordinates"]]

        add_polyline(
            map_obj=folium_map,
            coordinates=coordinates,
            color=colors[i % len(colors)],
            popup_html=get_popup_html(leg=leg, i=i)
        )

end_coords = legs[-1]["steps"][-1]["geometry"]["coordinates"][-1]
end_point = Point(latitude=end_coords[1], longitude=end_coords[0])
create_marker(
    map_obj=folium_map,
    point=end_point,
    label=len(legs) + 1,
    background_color="#ff6a6a",
    y_offset=0
)

folium_map